In [ ]:
!pip install librosa numpy scikit-learn matplotlib tqdm
import os
import librosa
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# configuration
DATA_DIR = 'fma_small/' # Change this
SAVE_DIR = 'processed_data/'
SAMPLE_RATE = 22050  # Standard for many music models
DURATION_SEC = 5     # Length of each chunk
N_MELS = 128         # Height of the spectrogram (frequency bins)

# 🔧 NOUVEAU: Limitation du nombre de dossiers à traiter
MAX_FOLDERS = 50     # Traiter seulement les 50 premiers dossiers (au lieu de tous)
                     # Mettez None pour traiter tous les dossiers

print(f"📊 Configuration:")
print(f"   - Dossiers à traiter: {'Tous' if MAX_FOLDERS is None else MAX_FOLDERS}")
print(f"   - Durée par segment: {DURATION_SEC} secondes")
print(f"   - Sample rate: {SAMPLE_RATE} Hz")
print(f"   - Mel bins: {N_MELS}")

os.makedirs(SAVE_DIR, exist_ok=True)

In [6]:
def create_mel_spectrogram(audio_chunk, sr, n_mels=128):
    # Compute Mel Spectrogram
    mel_spec = librosa.feature.melspectrogram(y=audio_chunk, sr=sr, n_mels=n_mels)
    
    # Convert power to decibels (log scale) - crucial for diffusion models
    log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
    
    # Normalize to [-1, 1] for the U-Net
    normalized_spec = (log_mel_spec + 40) / 40 # Adjust based on your DB range
    return normalized_spec

In [ ]:
# 📊 Analyse du dataset avant traitement
print("🔍 Analyse du dataset FMA:")

total_folders = 0
total_mp3_files = 0

for root, dirs, files in os.walk(DATA_DIR):
    mp3_files = [f for f in files if f.endswith('.mp3')]
    if mp3_files:
        total_folders += 1
        total_mp3_files += len(mp3_files)
        
        if total_folders <= 5:  # Affiche les 5 premiers dossiers
            print(f"   📁 {os.path.basename(root)}: {len(mp3_files)} fichiers MP3")

print(f"\n📈 Statistiques totales:")
print(f"   - Dossiers avec MP3: {total_folders}")
print(f"   - Fichiers MP3 total: {total_mp3_files}")

if MAX_FOLDERS is not None and MAX_FOLDERS < total_folders:
    print(f"\n⚡ Traitement limité à {MAX_FOLDERS} dossiers sur {total_folders}")
    estimated_files = int((total_mp3_files / total_folders) * MAX_FOLDERS)
    print(f"   - Estimation fichiers traités: ~{estimated_files}")
else:
    print(f"\n🚀 Traitement complet de tous les {total_folders} dossiers")

In [ ]:
all_spectrograms = []
folders_processed = 0

# Walk through directories avec limitation
for root, dirs, files in os.walk(DATA_DIR):
    # Vérification de la limitation de dossiers
    if MAX_FOLDERS is not None and folders_processed >= MAX_FOLDERS:
        print(f"🛑 Limite atteinte: {MAX_FOLDERS} dossiers traités")
        break
    
    # Traiter seulement si le dossier contient des fichiers MP3
    mp3_files = [f for f in files if f.endswith('.mp3')]
    if not mp3_files:
        continue
        
    print(f"📁 Traitement dossier {folders_processed + 1}: {os.path.basename(root)} ({len(mp3_files)} fichiers MP3)")
    folders_processed += 1
    
    for file in tqdm(mp3_files, desc=f"Dossier {folders_processed}", leave=False):
        file_path = os.path.join(root, file)
        try:
            # Load audio
            y, sr = librosa.load(file_path, sr=SAMPLE_RATE)
            
            # Calculate samples per chunk
            samples_per_chunk = DURATION_SEC * SAMPLE_RATE
            
            # Create chunks
            for i in range(0, len(y) - samples_per_chunk, samples_per_chunk):
                chunk = y[i : i + samples_per_chunk]
                spec = create_mel_spectrogram(chunk, SAMPLE_RATE, N_MELS)
                all_spectrograms.append(spec)
                
        except Exception as e:
            print(f"❌ Erreur traitement {file}: {e}")

# Convert list to a large numpy array [N, Channels, H, W]
# We add a channel dimension (1) so the U-Net treats it like a grayscale image
dataset = np.array(all_spectrograms)[:, np.newaxis, :, :]
print(f"✅ Total spectrogrammes créés: {dataset.shape}")
print(f"📁 Dossiers traités: {folders_processed}")
print(f"🎵 Segments de {DURATION_SEC}s: {len(all_spectrograms)}")

 17%|█▋        | 15/87 [00:01<00:09,  7.69it/s]



KeyboardInterrupt: 

In [ ]:
# First split: Train vs Temp (20%)
x_train, x_temp = train_test_split(dataset, test_size=0.2, random_state=42)

# Second split: Split Temp into Val (50%) and Test (50%)
x_val, x_test = train_test_split(x_temp, test_size=0.5, random_state=42)

print(f"Train: {x_train.shape}")
print(f"Val: {x_val.shape}")
print(f"Test: {x_test.shape}")

In [ ]:
# Save as numpy files
np.save(os.path.join(SAVE_DIR, 'train_specs.npy'), x_train)
np.save(os.path.join(SAVE_DIR, 'val_specs.npy'), x_val)
np.save(os.path.join(SAVE_DIR, 'test_specs.npy'), x_test)

# Visualize a random sample to check
plt.figure(figsize=(10, 4))
plt.imshow(x_train[0, 0], aspect='auto', origin='lower')
plt.title("Sample Mel Spectrogram")
plt.colorbar(format='%+2.0f dB')
plt.show()

In [4]:
# Test rapide pour calculer les dimensions
import librosa
test_audio = np.random.randn(5 * 22050)  # 5 secondes de test
test_spec = create_mel_spectrogram(test_audio, 22050, 128)
print(f"Dimensions du spectrogramme: {test_spec.shape}")
print(f"Format final avec channel: [1, {test_spec.shape[0]}, {test_spec.shape[1]}]")

Dimensions du spectrogramme: (128, 216)
Format final avec channel: [1, 128, 216]


# 🔧 Configuration du Preprocessing

Vous pouvez facilement ajuster les paramètres :

## Limitation de dossiers
- **`MAX_FOLDERS = 50`** : Traite seulement les 50 premiers dossiers (rapide pour tests)
- **`MAX_FOLDERS = None`** : Traite tous les dossiers (preprocessing complet)

## Autres paramètres
- **`DURATION_SEC = 5`** : Durée en secondes par segment audio
- **`SAMPLE_RATE = 22050`** : Fréquence d'échantillonnage 
- **`N_MELS = 128`** : Nombre de bins mel dans le spectrogramme

### Exemples d'usage :
- **Test rapide** : `MAX_FOLDERS = 10` (quelques minutes)
- **Dataset moyen** : `MAX_FOLDERS = 50` (recommandé)
- **Dataset complet** : `MAX_FOLDERS = None` (plusieurs heures)